In [ ]:
import numpy as np
import sys
import os
import pandas as pd
import plotly.graph_objects as go

sys.path.append(os.path.abspath("../libs"))
sys.path.append(os.path.abspath("../utils"))

from levenberg_marquadt import levenberg_marquadt
from normalize import  MinMaxNormalizer
from loss_fn_tarefa5 import make_mse_loss_for_network
from activations_fn import tanh_derivative
from network import hidden_forward, unflatten_weights, make_jacobian_fn, make_residuals_fn

pd.set_option('display.float_format', '{:.5f}'.format)

# Ajuste de curva por otimização

## Carregar os dados

In [9]:
# Carregamento dos dados
df = pd.read_excel('../data/Trabalho4dados.xlsx')

x1_data, x2_data = df['x1'], df['x2']

df_pivot = df.pivot(index='x2', columns='x1', values='y')

fig = go.Figure()

# Adicionar os Pontos de Dados Originais
fig.add_trace(go.Scatter3d(
    x=df['x1'],
    y=df['x2'],
    z=df['y'],
    mode='markers',
    marker=dict(size=3, color='red', symbol='circle'),
    name='Pontos de Dados Originais'
))

# Melhorar o Layout
fig.update_layout(
    title=dict(text='y = f(x1, x2)', x=0.5),
    scene=dict(
        xaxis_title='Eixo X1',
        yaxis_title='Eixo X2',
        zaxis_title='Eixo Y (Valor)'
    ),
    margin=dict(l=0, r=0, b=0, t=50)
)

fig.show()

## Calcular as funções de perda

### Pré-processamento dos dados

In [10]:
features = df[['x1', 'x2']]
y = df['y']

# Criar os objetos para Padronização
scaler = MinMaxNormalizer(-1, 1)
scaler_y = MinMaxNormalizer(-1, 1)

# Cria as cópias dos dados para padronizar
scaled_features = features.copy()
scaled_y = y.copy()

# Ajusta os padronizadores aos dados
scaler.fit(scaled_features)
scaler_y.fit(scaled_y.to_frame())

# Padroniza os dados
scaled_features = scaler.normalize(scaled_features)
scaled_y = scaler_y.normalize(scaled_y.to_frame()).squeeze()

# Preparar dados
x1_scaled = scaled_features['x1'].values
x2_scaled = scaled_features['x2'].values
y_scaled = scaled_y.values

# Definir parâmetros da rede neural
n_neurons = 2
n_iterations = 10000
tolerance = 1e-6
alpha = 1e-3

# Inicializar listas para armazenar os pesos
neurons_weights = []
neurons_raw_weights = []

# Inicializar pesos para cada neurônio (2 ocultos + 1 de saída)
# Cada neurônio oculto tem 3 pesos (x1, x2, bias)
hidden_weights = [np.random.randn(3) for _ in range(n_neurons)]
# Neurônio de saída tem n_neurons + 1 pesos (saídas dos neurônios ocultos + bias)
output_weights = np.random.randn(n_neurons + 1)

# Combinar todos os pesos em uma lista
initial_weights_list = hidden_weights + [output_weights]

# Concatenar todos os pesos em um único vetor para o Levenberg-Marquardt
initial_weights_flat = np.concatenate([w.flatten() for w in initial_weights_list])


### Inicializar as funções de perda e gradiente

In [11]:
# Função de custo e gradiente
loss_function, _ = make_mse_loss_for_network(x1_scaled, x2_scaled, y_scaled, activation_fn=np.tanh)

# Função de resíduos e jacobiana
residuals_fn = make_residuals_fn(
    x1_scaled,
    x2_scaled,
    y_scaled,
    n_neurons=n_neurons,
    activation_fn=np.tanh
)
jacobian_fn = make_jacobian_fn(
    x1_scaled,
    x2_scaled,
    n_neurons=n_neurons,
    activation_fn=np.tanh,
    activation_deriv=tanh_derivative
)

### Rodar os experimentos

In [ ]:
print(f"Treinando rede com {n_neurons} neurônios na camada oculta...")

# Treinar com Levenberg-Marquardt
weights_flat, losses, n_iters = levenberg_marquadt(
    initial_weights_flat, residuals_fn, loss_function, jacobian_fn,
    alpha=alpha, alpha_variability=10, max_iter=n_iterations,
    tolerance=tolerance, stopping_criteria=[1, 3]
)

# Usar os pesos finais
final_weights_flat = weights_flat[-1]

# Reconstruir os pesos para cada neurônio
idx = 0
for i in range(n_neurons):
    w_hidden = final_weights_flat[idx:idx+3]
    neurons_raw_weights.append(w_hidden)
    idx += 3

w_output = final_weights_flat[idx:]
neurons_raw_weights.append(w_output)

# Armazenar os pesos normalizados para uso na predição
neurons_weights = neurons_raw_weights.copy()

print(len(neurons_weights), [len(w) for w in neurons_weights])
# Fazer previsão usando a rede treinada
y_hat_scaled = hidden_forward(
    x1_scaled,
    x2_scaled,
    neurons_weights=neurons_weights,
    activation_fn=np.tanh
)
# Desnormalizar as previsões
# Convertendo para DataFrame com o mesmo índice usado no treinamento
y_hat_df = pd.DataFrame(y_hat_scaled.reshape(-1, 1), index=y.index)
y_hat = scaler_y.denormalize(y_hat_df).flatten()

Treinando rede com 2 neurônios na camada oculta...
3 [3, 3, 3]


## Resultados

In [13]:
# Calcular métricas
mse_final = np.mean((y - y_hat) ** 2)
rmse_final = np.sqrt(mse_final)
mae_final = np.mean(np.abs(y - y_hat))

# Preparar resultados para exibição
dict_results = {
    'Feature_Set': "MinMax(-1,1)",
    'Loss_Function': "MSE",
    'Final_Loss': losses[-1],
    'MSE_Final': mse_final,
    'RMSE_Final': rmse_final,
    'MAE_Final': mae_final,
    'Iterations': n_iters
}

# Adiciona os pesos de cada neurônio em colunas separadas
for i, weights in enumerate(neurons_weights):
    dict_results[f'Pesos_Neuronio_{i+1}'] = [float(f'{w:.6f}') for w in weights]

df_result = pd.DataFrame([dict_results])
df_result

,Feature_Set,Loss_Function,Final_Loss,MSE_Final,RMSE_Final,MAE_Final,Iterations,Pesos_Neuronio_1,Pesos_Neuronio_2,Pesos_Neuronio_3
0,"MinMax(-1,1)",MSE,0.00474,0.00522,0.07222,0.04102,14,"[-2.113904, -2.080318, -1.109331]","[-0.645363, -1.777609, -1.188523]","[-2.950495, 1.267199, 0.260162]"


In [ ]:
# Gere a malha para a superfície
x1_range = np.linspace(df['x1'].min(), df['x1'].max(), 30)
x2_range = np.linspace(df['x2'].min(), df['x2'].max(), 30)
x1_grid, x2_grid = np.meshgrid(x1_range, x2_range)

# Normalize a malha com o mesmo scaler usado no treino
x1_grid_flat = x1_grid.flatten()
x2_grid_flat = x2_grid.flatten()
mesh_df = pd.DataFrame({'x1': x1_grid_flat, 'x2': x2_grid_flat})
mesh_scaled = scaler.normalize(mesh_df)
x1_mesh_scaled = mesh_scaled['x1'].values
x2_mesh_scaled = mesh_scaled['x2'].values

# Predição na malha
y_grid_scaled = hidden_forward(
    x1_mesh_scaled, x2_mesh_scaled, neurons_weights=neurons_weights, activation_fn=np.tanh
)
y_grid_pred = scaler_y.denormalize(pd.DataFrame(y_grid_scaled.reshape(-1, 1))).flatten()
y_grid_pred = y_grid_pred.reshape(x1_grid.shape)

fig = go.Figure()

# Superfície de predição
fig.add_trace(go.Surface(
    x=x1_grid, y=x2_grid, z=y_grid_pred,
    colorscale='Blues', opacity=0.7, showscale=False, name='Superfície de Predição'
))

# Pontos originais
fig.add_trace(go.Scatter3d(
    x=df['x1'], y=df['x2'], z=df['y'],
    mode='markers',
    marker=dict(size=7, color='red', symbol='circle', line=dict(width=1, color='darkred')),
    name='Dados Originais'
))

# Pontos previstos
fig.add_trace(go.Scatter3d(
    x=df['x1'], y=df['x2'], z=y_hat,
    mode='markers',
    marker=dict(size=7, color='blue', symbol='diamond', line=dict(width=1, color='darkblue')),
    name='Valores Previstos'
))

# Linhas verticais
for i in range(len(df)):
    fig.add_trace(go.Scatter3d(
        x=[df['x1'].iloc[i], df['x1'].iloc[i]],
        y=[df['x2'].iloc[i], df['x2'].iloc[i]],
        z=[df['y'].iloc[i], y_hat[i]],
        mode='lines',
        line=dict(color='rgba(0,100,0,0.5)', width=2),
        showlegend=False
    ))

# Layout
fig.update_layout(
    title='Modelo do Neurônio: Valores Reais vs. Previstos',
    scene=dict(
        xaxis_title='x1',
        yaxis_title='x2',
        zaxis_title='y',
        aspectmode='cube',
        camera=dict(
            eye=dict(x=1.5, y=-1.5, z=1),
            up=dict(x=0, y=0, z=1)
        ),
        xaxis=dict(gridcolor='lightgray'),
        yaxis=dict(gridcolor='lightgray'),
        zaxis=dict(gridcolor='lightgray')
    ),
    width=1000,
    height=800,
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=0.01,
        bgcolor="rgba(255,255,255,0.8)",
        bordercolor="black",
        borderwidth=1
    )
)

fig.show()